## BOAR for TrPL fitting
Version 0.1
(c) Larry Lueer, Vincent M. Le Corre, i-MEET 2021-2023

This notebook is made to use BOAR for fitting TrPL data with rate equations.

Here we use the following rate equations:

$$\frac{dn}{dt}  = G - k_{trap} n (Bulk_{tr} - n_t) - k_{direct} * n * (p + p_0)$$
$$\frac{dn_t}{dt}= k_{trap} * n * (Bulk_{tr} - n_t) - k_{detrap} * n_t * (p + p_0)$$
$$\frac{dp}{dt} = G - k_{detrap} * n_t * (p + p_0) - k_{direct} * n * (p + p_0)$$

where $n$ and $p$ are the electron and hole charge carrier densities, $G$ is the generation rate in m<sup>-3</sup> s<sup>-1</sup>, k<sub>trap</sub> and k<sub>detrap</sub> are the trapping and detraping rates in m<sup>3</sup> s<sup>-1</sup>, and k<sub>direct</sub> is the bimolecular/band-to-bad recombination rate in m<sup>3</sup> s<sup>-1</sup>.


In [ ]:
# Activate matplotlib widgets
%matplotlib inline
# comment the next line if you are on the jupyterhub server
%matplotlib widget 
# %matplotlib notebook

# Import libraries
import sys
from numpy.random import default_rng
from scipy import interpolate
from sklearn.preprocessing import minmax_scale
import warnings
warnings.filterwarnings('ignore') # comment this out to see warnings

# Import boar
# import one folder up
sys.path.append('../') # comment out if the Notebook is in the Notebooks folder
from boar import *
# Import homemade package by VLC
# import boar.SIMsalabim_utils.plot_settings_screen # to set default plot settings

In [ ]:
# Define the path to the data and to the SIMsalabim directory
curr_dir = os.getcwd()
res_dir = os.path.join(curr_dir,'temp') # path to the results directory, use this line if Notebook is in the Notebooks folder
# res_dir = os.path.join('../','temp') # path to the results directory, use this line if Notebook is in the Notebooks folder



In [ ]:
# define Fitparameters
# create fit parameters for the various processes we have to consider
# True_values = {'ktrap': 2e-11, 'kdirect': 1e-14, 'kdetrap':0e-17,'Bulk_tr':1e20,'p_0':2e19, 'QE': 0.9, 'I_MC': 1e-19, 'N0': 2.25e21, 'r_mu':1}
True_values = {'ktrap': 1e-15, 'kdirect': 2e-18, 'kdetrap':1e-17,'Bulk_tr':1e18,'p_0':1e14, 'QE': 0.9, 'I_MC': 1e-19, 'N0': 1e22, 'r_mu':1}

params = []
ktrap = Fitparam(name = 'ktrap', val = True_values['ktrap'] , relRange = 1, range_type = 'log',
            lim_type = 'relative',optim_type='log', display_name = 'k$_{trap}$ [m$^{3}$ s$^{-1}$]')
params.append(ktrap)
kdirect = Fitparam(name = 'kdirect', val =  True_values['kdirect'], relRange = 1, range_type = 'log',
            lim_type = 'relative',optim_type='log', display_name = 'k$_2$ [m$^{3}$ s$^{-1}$]')
params.append(kdirect)
kdetrap = Fitparam(name = 'kdetrap', val =  True_values['kdetrap'], relRange = 1, range_type = 'log',
            lim_type = 'relative',optim_type='log', display_name = 'k$_{detrap}$ [m$^{3}$ s$^{-1}$]')
params.append(kdetrap)
Bulk_tr = Fitparam(name = 'Bulk_tr', val =  True_values['Bulk_tr'], relRange = 1, range_type = 'log',
            lim_type = 'relative',optim_type='log', display_name = 'N$_T$ [m$^{-3}$]')
params.append(Bulk_tr)
p_0 = Fitparam(name = 'p_0', val =  True_values['p_0'], relRange = 1, range_type = 'log',
            lim_type = 'relative',optim_type='log', display_name = 'p$_0$ [m$^{-3}$]')
params.append(p_0)
QE = Fitparam(name = 'QE', val =  True_values['QE'], lims = [0.02, 1], relRange = 0, range_type = 'linear',
            lim_type = 'absolute',optim_type = 'linear', display_name = 'QE [%]')
params.append(QE)
I_MC = Fitparam(name = 'I_MC', val =  True_values['I_MC'], lims = [1e-18, 1e-16], relRange = 0, range_type = 'linear',
            lim_type = 'absolute',optim_type = 'linear', axis_type='log',display_name= 'I$_{PL}$')
params.append(I_MC)
N0 = Fitparam(name = 'N0', val =  True_values['N0'], lims = [5e21, 5e22], relRange = 0, range_type = 'log',
            lim_type = 'absolute',optim_type = 'log', axis_type='log',display_name= 'N$_0$ [m$^{-3}$]')
params.append(N0)
r_mu = Fitparam(name = 'r_mu', val =  True_values['r_mu'], lims = [1e-1, 10], relRange = 0, range_type = 'log',
            lim_type = 'absolute',optim_type = 'log', axis_type='log',display_name= 'r$_{mu}$')
params.append(r_mu)


In [ ]:
# Make fake data
rng = default_rng()
noise = 1e3 # noise level
fpu = 1000 # pump frequency in Hz
background = 0e24 # background pump density in m^-3

# create a pump pulse
x1 = np.linspace(0,0.99*1/fpu,10000)
X_dimensions = ['t','Gfrac']

X,y,weight = [],[],[]
trMC = TrMC_agent()
trMC.trMC_model = Bimolecular_Trapping_Detrapping_equation # set the model
trMC.pump_params['fpu'] = fpu
trMC.pump_params['background'] = background

Gfracs = np.geomspace(1e-3,1,endpoint=True,num=5)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,10))
for Gfrac in Gfracs:
    X1 =  [[xx, Gfrac] for xx in x1]
    X = X +  X1
    y1 = trMC.trMC(np.asarray(X1), params, X_dimensions=X_dimensions) #+ rng.standard_normal(len(X1))*noise
    ax1.plot(x1*1e9,y1/max(y1),label=f'{Gfrac:.2e}')
    ax2.semilogy(x1*1e9,y1)
    y = y + list(np.log10(y1))
    weight = weight + list(1/(np.abs(max(y1)*np.ones(len(y1))))) #


# ax1.set_xlim([0,100])
# ax1.set_ylim([0,1.1])
ax1.set_ylabel('$\sigma$ [a.u.]')
ax1.set_xlabel('Time [$\mu$s]')
ax1.legend(title='Gfrac')
# ax2.set_xlim([0,3])
# ax2.set_ylim([5e2,2e9])
ax2.set_ylabel('$\sigma$ [a.u.]')
ax2.set_xlabel('Time [$\mu$s]')
plt.tight_layout()

X = np.array(X)
y = np.array(y)
weight = np.array(weight)
weight/=np.mean(weight)
weight[weight>10]=10    


In [ ]:
# start multiobjective optimization
X_dimensions = ['t','Gfrac']
y_dimension =  '$\sigma$ [a.u.]'
target = {'model':partial(trMC.trMC,X_dimensions=X_dimensions,take_log=True), 'target_name':'trMC',
          'data':{'X':X,'y':y,'X_dimensions':X_dimensions,'X_units':['s',''],'y_dimension':y_dimension,'y_unit':''}
            ,'weight':weight,'target_weight':1}

targets = [target]
mo = MultiObjectiveOptimizer(targets = targets, params=params, res_dir=res_dir)

# define the optimization parameters
n_jobs = 4 
n_jobs_init = 20 
n_yscale=20 
n_BO = 60
n_initial_points = 60 
n_BO_warmstart = 80 

mo.warmstart = 'None' # whether to use warmstart to recall or collect points or not
mo.SaveOldXY2file = os.path.join(res_dir,'old_XY.json') # path to the file where old points are saved
mo.Path2OldXY = os.path.join(res_dir,'old_XY.json') # path to the file where old points are saved
kwargs = {'check_improvement':'relax','max_loop_no_improvement':10,'xtol':1e-3,'ftol':1e-3}
kwargs_posterior = {'Nres':4,'gaussfilt':3,'logscale':True,'vmin':1e-100,'zoom':0,'min_prob':1e-40,'clear_axis':False,'show_points':True,'savefig':True,'figname':'param_posterior','True_values':True_values}
kwargs_plot_obj = {'zscale':'linear'}

r = mo.optimize_sko_parallel(n_jobs=n_jobs,n_yscale=n_yscale, n_BO=n_BO, n_initial_points = n_initial_points,n_BO_warmstart=n_BO_warmstart,n_jobs_init=n_jobs_init,kwargs=kwargs,verbose=False,loss='linear',threshold=1000,base_estimator = 'GP',show_objective_func=False,show_posterior=True,kwargs_posterior = kwargs_posterior,kwargs_plot_obj=kwargs_plot_obj)



In [ ]:
# Plot and save the results
fit_results = []
kwargs_plot_res = {'x_scaling':1e6,'xaxis_label':'Time [$\mu$s]','xscale_type':'linear','yscale_type':'log','norm_data':False,'delog':False,'figsize':(10,10)}

for num,t in enumerate(targets):
    kwargs_plot_res['figname'] = os.path.join(res_dir,t['target_name']+f'_fit_{num}')
    trMC.plot_fit_res(t,mo.params,'t',xlim=[0,100],ylim=[],kwargs=kwargs_plot_res)

    X = t['data']['X']
    y = t['data']['y']
    X_dimensions = t['data']['X_dimensions']
    yfit = t['model'](X,params,X_dimensions=X_dimensions) # get the best fits

    data = np.concatenate((X, y.reshape(len(y),1), yfit.reshape(len(yfit),1)), axis=1)
    fit_results.append(data)

# prepare the data for saving
param_dict = trMC.get_param_dict(mo.params) # get fitparameters (and fixed ones)

pout = [[f'{v:.3E}' if isinstance(v,float) else v for _,v in pp.items()] for pp in param_dict]


# produce output excel file with data, fitparameters and FOMs
fn_xlsx = 'fits_results.xlsx'
namecols = X_dimensions + ['Jexp','Jfit']
# delete old file if it exists
if os.path.exists(os.path.join(res_dir,fn_xlsx)):
    os.remove(os.path.join(res_dir,fn_xlsx))

with pd.ExcelWriter(os.path.join(res_dir,fn_xlsx), mode='w') as writer:
    for i,t in enumerate(targets):
        if 'target_name' in t.keys():
            tname = t['target_name']
        else: 
            tname = 'data'
        namecols = X_dimensions + [tname+'_exp',tname+'_fit']
        df = pd.DataFrame(fit_results[i],columns=namecols)
        df.to_excel(writer, sheet_name = tname+f'_{i}')
       
    df = pd.DataFrame(pout,columns=[k for k in param_dict[0].keys()])
    df.to_excel(writer, sheet_name = f'params')


In [ ]:
# Clean output files from simulation folders
from boar.SIMsalabim_utils.CleanFolder import *
Do_Cleaning = False # Careful, this will delete all files in the folder
if Do_Cleaning:
    # delete old_xy.json file if it exists
    # os.remove(mo.path2oldxy) # remove the old_xy.json file if it exists
    # delete warmstart folder if it exists
    if os.path.exists(os.path.join(os.getcwd(),'warmstart/')):
        shutil.rmtree(os.path.join(os.getcwd(),'warmstart/'))
    # delete temp folder if it exists
    if os.path.exists(os.path.join(os.getcwd(),'temp/')):
        shutil.rmtree(os.path.join(os.getcwd(),'temp/'))
